## 4.2 激活函数—加入非线性因素，解决线性模型缺陷

In [1]:
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(-10, 10, 1000)
# Sigmoid函数
sigmoid = 1/(1+np.exp(-x))

# Tanh函数
tanh = (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))

# Relu函数
relu = np.where(x > 0, x, 0)

# Softplus函数
softplus = np.log(1+np.exp(x))

# Noisy relus函数
noisy_relu = np.where(x > 0, x+np.random.normal(loc=0, scale=np.std(x)), 0)

# Leaky relus函数
leaky_relu = np.where(x > 0, x, 0.1*x)

# Elus函数
elus = np.where(x>0, x, (0.1 * np.exp(x)-1))

# Swish函数
swish = (x * sigmoid)

loss_func = [sigmoid, tanh, relu, softplus, noisy_relu, leaky_relu, elus, swish]
loss_func_tf=['sigmoid',
              'tanh',
              'relu',
              'softplus',
              'noisy_relu',
              'leaky_relu',
              'elus',
              'swish']

fig, axes = plt.subplots(4, 2, figsize=(5, 10))
fig.tight_layout()

for loss_func, loss_func_tf, ax in zip(loss_func, loss_func_tf, axes.ravel()):
    ax.set_title(loss_func_tf)
    ax.grid()
    ax.plot(x, loss_func)
plt.show()

plt.figure(figsize=(5,5))
plt.grid()
plt.plot(x, sigmoid, label='sigmoid')
plt.plot(x, tanh, label='tanh')
plt.plot(x, relu, label='relu')
plt.plot(x, softplus, label='softplus')
plt.plot(x, leaky_relu, label='leaky_relu')
plt.plot(x, elus, label='elus')
plt.plot(x, swish, label='swish')
plt.legend()
plt.show()

<Figure size 500x1000 with 8 Axes>

<Figure size 500x500 with 1 Axes>

## 4.5 softmax算法与损失函数的综合应用

### 4.5.1 交叉熵实验

In [11]:
# 假设有一个标签labels和一个网络输出值logits
# 两次softmax实验: 将输出值logits分别进行一次和两次softmax，观察两次的区别和意义
# 观察交叉熵：将上一步中的两个值分别进行softmax_corss_entropy_with_logits,观察它们的区别
# 自建公式实验：将两次softmax的值放入自建组合的公式中得到正确的值

import tensorflow as tf

# labels and logits
labels = [[0, 0, 1], [0, 1, 0]]
logits = [[2, 0.5, 6],
          [0.1, 0, 3]]

logits_scaled = tf.nn.softmax(logits) # 真实转化的softmax值
logits_scaled2 = tf.nn.softmax(logits_scaled) # 经过第二次softmax后，分布概率会有变化

# 正确的方式
result1 = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)

# 传入softmax_cross_entropyIwith_logits的logits不需要进行softmax
# 如果将softmax变换完的值放进去，就相当于算第二次softmax的loss，所以会出错
result2 = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits_scaled)

# 对于已经用softmax转换过的scaled，可以自建一个loss函数
result3 = -tf.reduce_sum(labels*tf.log(logits_scaled), axis=1)

with tf.Session() as sess:
    print('scaled={}'.format(sess.run(logits_scaled)))
    print('scaled2={}'.format(sess.run(logits_scaled2)))
    print('rel1={}'.format(sess.run(result1)))
    print('rel2={}'.format(sess.run(result2)))
    print('rel3={}'.format(sess.run(result3)))

# logits中的值原本加和是大于1的，经过softmax后，总和变成了1
# 样本中第一个是跟标签分类相符的， 第二与标签分类不符，所以第一个的交叉熵比较小，第二个比较大


scaled=[[0.01791432 0.00399722 0.97808844]
 [0.04980332 0.04506391 0.90513283]]
scaled2=[[0.21747023 0.21446465 0.56806517]
 [0.2300214  0.22893383 0.5410447 ]]
rel1=[0.02215516 3.0996735 ]
rel2=[0.56551915 1.4743223 ]
rel3=[0.02215518 3.0996735 ]



### 4.5.2 one_hot实验

In [14]:
# 输入的标签也可以不是标准的one_hot
# 下面用一组总和是1，但数组中每个值不等于0或1的数组来代替标签

# 对非one_hot编码为标签的数据进行交叉熵的计算，比较其与one_hot编码之间的差别

labels = [[0.4, 0.1, 0.5],
          [0.3, 0.6, 0.1]]
result4 = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
with tf.Session() as sess:
    print('rel4={}'.format(sess.run(result4))) 
    # 对比rel1发现，正确分类的交叉熵和错误分类的交叉熵，二者的错误没有标准one_hot明显

rel4=[2.1721554 2.7696736]


### 4.5.3 sparse交叉熵的使用

In [16]:
# 使用sparse_softmax_cross_with_logits函数对非one_hot的标签进行交叉熵计算，比较其与one_hot标签的区别
# sparse_softmax_cross_with_logits需要使用非one_hot的标签

# sparse标签
labels =[2,1] # 表明labels中总共分为3个类： 0、1和2
              # 2、1分别对应one_hot编码中的001与010
result5 = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
with tf.Session() as sess:
    print('rel5={}'.format(sess.run(result5))) # 与rel1的结果相同

rel5=[0.02215516 3.0996735 ]


### 计算loss值

In [17]:
loss=tf.reduce_sum(result1)
with tf.Session() as sess:
    print('loss={}'.format(sess.run(loss)))

loss=3.121828556060791


In [19]:
labels=[[0,0,1],[0,1,0]]
loss2 = -tf.reduce_sum(labels * tf.log(logits_scaled))
with tf.Session() as sess:
    print('loss2={}'.format(sess.run(loss2))) # 与loss值相同

loss2=3.12182879447937


## 6.6 梯度下降—让模型逼近最小偏差

### 6.6.1 梯度下降的作用及分类

In [20]:
# 常用的梯度下降方法
# 批量梯度下降
# 随机梯度下降
# 小批量梯度下降

### 6.6.2 TensorFlow中的梯度下降函数

In [21]:
# tf.train.GradientDescentOptimizer()
# tf.train.AdadeltaOptimizer()
# tf.train.AdagradOptimizer()
# tf.train.MomentumOptimizer()
# tf.train.AdamOptimizer()
# tf.train.FtrlOptimizer()
# tf.train.RMSPropOptimzer()

# 在训练过程中，先实例化一个优化函数，并基于一定的学习率进行梯度优化训练
# 然后使用一个minimize()的操作，里面传入损失值节点loss
# 再启动一个外层的循环，优化器就会按照循环的次数沿着loss的最小值的方向优化参数了

### 6.6.3 退化学习率—在训练的速度与精度之间找到平衡

In [24]:
# 每个优化器的第一个参数learning_rate就是代表学习率
# 设置学习率的大小，是在精度和速度之间找到一个平衡:
# 如果学习率的值比较大，则训练的速度会提升，但结果精度不够
# 如果学习率的值比较小，精度虽然提升了，但训练会耗费太多时间

# 退化学习率在训练刚开始时使用大的学习率加快速度，训练到一定程度后使用小的学习率来提高精度

# 学习率的衰减是由global_step和decay_steps来决定的
# def exponential_decay(learning_rate,global_step,decay_steps,decay_rate, staircase=False, name=None):
#     decayed_learning_rate = learning_rate * decay_rate ^ (global_step / decay_steps)
# staircase值默认为False，当为True时，将没有衰减功能

# learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 100000, 0.96)
# 当前迭代到global_step步，学习率每一步都按照每100000步缩小到0.96的速度衰减

# 有时还需要对已经训练好的模型进行微调，可以指定不同层使用不同的学习率
# 增大批次处理样本的数量也可以起到退化学习率的作用，但这种方法要求训练时的最小批次要与实际应用中的最小批次一致


### 6.6.4 退化学习率用法举例

In [31]:
# 定义一个学习率变量，将其衰减系数设置好，并设置好循环次数
# 将每次迭代运算的次数与学习率打印出来，观察学习率按照次数退化的现象

# 初始学习率为0.1， 令其以每10次衰减到0.9倍的速度进行退化
import tensorflow as tf
global_step = tf.Variable(0, trainable=False)
initial_learning_rate=0.1
learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step=global_step, 
                                           decay_steps=10, decay_rate=0.9)
opt = tf.train.GradientDescentOptimizer(learning_rate)
add_global = global_step.assign_add(1) # 定义一个op，令global_step加1完成计步
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(learning_rate))
    for i in range(20):
        g, rate = sess.run([add_global, learning_rate])
        print(g, rate)

0.1
1 0.1
2 0.09895193
3 0.09791484
4 0.09688862
5 0.095873155
6 0.094868325
7 0.092890166
8 0.092890166
9 0.09191661
10 0.089999996
11 0.08905673
12 0.08812335
13 0.08812335
14 0.08628584
15 0.08628584
16 0.08448663
17 0.08448663
18 0.08272495
19 0.08272495
20 0.08099999


## 6.7 初始化学习参数

In [33]:
# 在定义学习参数时可以通过get_variable和Variable两个方式

## 6.8 单个神经元的扩展—Maxout网络

### 6.8.1 Maxout介绍

In [34]:
# Maxout网络可以理解为单个神经元的扩展，主要是扩展单个神经元的激活函数
# Maxout是将激活函数变成一个网络选择器，原理是将多个神经元并列地放在一起
# 从它们的输出结果中找出最大的那个，代表对特征响应最敏感，然后取这个神经元参与后面的计算

### 6.8.2 用Maxout网络实现MNIST分类

In [ ]:
# Maxout网络的构建方法: 通过reduce_max函数对多个神经元的输出来计算Max值
# 将Max值作为输入按照神经元正反向传播进行计算

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import pylab

mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
tf.reset_default_graph()
# 定义占位符
x = tf.placeholder(tf.float32, shape=[None, 28*28]) 
y = tf.placeholder(tf.float32, shape=[None, 10]) 
# 定义学习参数
w = tf.Variable(tf.random_normal(([784, 10])), name='weight')
b = tf.Variable(tf.zeros(([10])), name='bias')
# 搭建正向模型—定义输出节点
z = tf.nn.softmax(tf.matmul(x, w) + b) # softmax分类

# Maxout
maxout = tf.reduce_max(z, axis=1, keep_dims=True)

# 设置学习参数
W2 = tf.Variable(tf.truncated_normal([1, 10], stddev=0.1))